In [1]:
# munging imports
import pandas as pd
import numpy as np
import sys
import pickle

# visualization imports
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
# modeling imports
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score,f1_score, roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer, fbeta_score

In [7]:
ls

BankChurners.csv              Porject Proposal.md
Churned_Prediction.ipynb      churned_data.pickle
Model Test By Segments.ipynb


In [49]:
#import pickle data
with open('churned_data.pickle', 'rb') as read_file:
    df = pickle.load(read_file)

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 47 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   CLIENTNUM                    10127 non-null  int64  
 1   Attrition_Flag               10127 non-null  object 
 2   Customer_Age                 10127 non-null  int64  
 3   Gender                       10127 non-null  object 
 4   Dependent_count              10127 non-null  int64  
 5   Education_Level              10127 non-null  object 
 6   Marital_Status               10127 non-null  object 
 7   Income_Category              10127 non-null  object 
 8   Card_Category                10127 non-null  object 
 9   Months_on_book               10127 non-null  int64  
 10  Total_Relationship_Count     10127 non-null  int64  
 11  Months_Inactive_12_mon       10127 non-null  int64  
 12  Contacts_Count_12_mon        10127 non-null  int64  
 13  Credit_Limit    

In [51]:
features = ['Customer_Age','Female','Dependent_count', 'College', 'Doctorate', 'Graduate', 'High School', 'Post-Graduate', 'Uneducated', 
            'Divorced', 'Married', 'Single', '$120K +', '40K− 60K', '60K− 80K', '80K− 120K', 'Less than $40K',
            'Blue', 'Gold','Platinum', 'Months_on_book', 'Total_Relationship_Count',
            'Total_Revolving_Bal','Total_Amt_Chng_Q4_Q1','Total_Trans_Amt','Total_Trans_Ct',
            'Total_Ct_Chng_Q4_Q1','Months_Inactive_12_mon', 'Contacts_Count_12_mon', 'Avg_Utilization_Ratio','Attrited Customer']

In [52]:
df = df[features]

In [53]:
df_m = df[df['Female'] == 1]
df_f = df[df['Female'] == 0]

In [54]:
X_m = df_m.drop(columns = ['Attrited Customer'])
y_m = df_m['Attrited Customer']

In [55]:
X_f = df_f.drop(columns = ['Attrited Customer'])
y_f = df_f['Attrited Customer']

In [56]:
with open('gbm_model.pickle', 'rb') as read_file:
    gbm = pickle.load(read_file)

In [57]:
#test on Male/Female group
def gender_test(features, target, func = gbm):
    acc, f2, auc, prec, rec = [],[],[],[],[]

    proba = func.predict_proba(features, ntree_limit=gbm.best_ntree_limit)[:,1]
    labels = proba >= 0.3

    acc= accuracy_score(target, labels)
    f2 = fbeta_score(target, labels, beta = 2)
    auc = roc_auc_score(target, proba)
    prec = precision_score(target, labels)
    rec = recall_score(target, labels)
    
    return acc, f2, auc, prec, rec

In [58]:
#Male group
gender_test(X_m, y_m, func = gbm)

(0.9781634938409854,
 0.9461193072482363,
 0.9967095997124845,
 0.9247648902821317,
 0.9516129032258065)

In [59]:
#female group
gender_test(X_f, y_f, func = gbm)

(0.9742084294401342,
 0.9221778791334092,
 0.9929292815405907,
 0.8986111111111111,
 0.9282639885222381)

#### model performance are similar in male and female group